In [1]:
# Imports
import functools
from multiprocessing import Pool
import opensmile
import pandas as pd
from pathlib import Path
import soundfile as sf

In [2]:
# Worker function
def featureWorker(audioPath, smile, label):
    df = None
    with audioPath.open('rb') as file:
        data, samplerate = sf.read(file)
        for i in range(int(data.shape[0]/samplerate/30)):
            name = audioPath.name.replace('.mp3', '').replace('.flac', '') + '_' + str(i)
            subData = data[i*samplerate*30:(i+1)*samplerate*30, 1]
            if df is None:
                df = smile.process_signal(subData, samplerate)
                df['name'] = [name]
                df['label'] = [label]
                df = df.set_index(['name', 'label'])
            else:
                tempDf = smile.process_signal(subData, samplerate)
                tempDf['name'] = [name]
                tempDf['label'] = [label]
                tempDf = tempDf.set_index(['name', 'label'])
                df = pd.concat([df, tempDf])
    return df

In [25]:
# Test worker function
# Only for debugging, do not run when generating the dataset
data, samplerate = sf.read('dataset/MP3_to_FLAC_insanely_high/001-Endwalker.mp3.flac')
df = None
smile = opensmile.Smile(
    feature_set = opensmile.FeatureSet.ComParE_2016,
    feature_level = opensmile.FeatureLevel.Functionals,
)
# Extract features for each 30 seconds
# Ignore snippets under 30 seconds
for i in range(int(data.shape[0]/samplerate/30)):
    filename = 'Test_' + str(i)
    subData = data[i*samplerate*30:(i+1)*samplerate*30, 1]
    if df is None:
        df = featureWorker(subData, samplerate, smile)
        df['name'] = [filename]
        df = df.set_index('name')
    else:
        print('Adding')
        tempDf = featureWorker(subData, samplerate, smile)
        tempDf['name'] = [filename]
        tempDf = tempDf.set_index('name')
        df = pd.concat([df, tempDf])
df

Adding
Adding
Adding
Adding


,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
name,,,,,,,,,,,,,,,,,,,,,
Test_0,18.277109,0.211671,15.964147,18.645599,20.490549,4.526402,162.687912,197.854248,113.263168,103.828979,...,-0.027175,-0.013031,4.122325,2.934311,2.242303,0.375522,0.556882,0.071136,0.063001,-14.361074
Test_1,18.932316,0.226265,15.517254,19.041876,22.269821,6.752567,161.841034,163.118103,117.282478,136.512329,...,-0.033687,-0.015582,6.106512,3.401134,2.440655,0.334658,0.391618,0.072453,0.053936,-9.601865
Test_2,18.917887,0.320817,15.583258,18.412197,19.159142,3.575884,308.761871,556.804077,167.305450,179.005753,...,-0.026049,-0.005922,6.681701,5.035012,2.805611,0.154048,0.200886,0.219855,0.299165,-8.946024
Test_3,18.820414,0.228321,16.858976,18.807341,19.145868,2.286892,121.434937,124.874252,160.076996,290.345245,...,-0.037341,-0.017632,8.209599,4.668223,4.209823,0.120079,0.114445,0.105826,0.105113,-8.319811
Test_4,19.925528,0.312412,17.017380,18.581041,20.152973,3.135593,163.498901,186.520416,209.951843,440.281647,...,-0.032492,-0.012355,8.315269,5.568523,4.243235,0.097402,0.080825,0.125714,0.155009,-8.117042


In [3]:
# Input paths
losslessPath = Path('dataset/All_44KHz_lossless')
lossyPath = Path('dataset/MP3_to_FLAC_insanely_high')
# Output paths
smileFeaturePath = Path('dataset/opensmile_features.csv')

# Construct dataset
df = None
# Initialize opensmile
smile = opensmile.Smile(
    feature_set = opensmile.FeatureSet.ComParE_2016,
    feature_level = opensmile.FeatureLevel.Functionals,
)

In [4]:
# Go through lossless files
featureLossless = functools.partial(featureWorker, smile=smile, label=0)

with Pool() as pool:
    results = pool.imap(featureLossless, losslessPath.iterdir(), chunksize=50)
    for resultDf in results:
        if df is None:
            df = resultDf
        else:
            df = pd.concat([df, resultDf])

In [58]:
df.index

MultiIndex([(                      '031-芒種、鼠黐の薫風_0', 0),
            (                      '031-芒種、鼠黐の薫風_1', 0),
            (                      '031-芒種、鼠黐の薫風_2', 0),
            (                      '031-芒種、鼠黐の薫風_3', 0),
            (                      '031-芒種、鼠黐の薫風_4', 0),
            (                      '031-芒種、鼠黐の薫風_5', 0),
            (                      '031-芒種、鼠黐の薫風_6', 0),
            (                      '031-芒種、鼠黐の薫風_7', 0),
            (                      '031-芒種、鼠黐の薫風_8', 0),
            (                      '031-芒種、鼠黐の薫風_9', 0),
            ...
            ( '046-遊園施設 (From NieR_Automata).mp3_4', 1),
            (                     '31_知恵を絞って.mp3_0', 1),
            (                     '31_知恵を絞って.mp3_1', 1),
            (                     '31_知恵を絞って.mp3_2', 1),
            (                     '31_知恵を絞って.mp3_3', 1),
            ('20 SPRING RIGHTEOUSNESS –首都・蜀–.mp3_0', 1),
            ('20 SPRING RIGHTEOUSNESS –首都・蜀–.mp3_1', 1),
            ('2

In [5]:
# Go through lossy files
featureLossy = functools.partial(featureWorker, smile=smile, label=1)

with Pool() as pool:
    results = pool.imap(featureLossy, lossyPath.iterdir(), chunksize=50)
    for resultDf in results:
        if df is None:
            df = resultDf
        else:
            df = pd.concat([df, resultDf])

In [6]:
# Write feature to file
df.to_csv(smileFeaturePath)